# Importing Libraries

In [16]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler, Subset
from torchvision import datasets, transforms
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision.models import resnet18
from sklearn.metrics import f1_score, classification_report

# Importing Dataset

In [18]:
root_dir = "dataset"
train_ratio = 0.8
batch_size = 32
num_workers = 4

train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(root=root_dir)
targets = np.array(full_dataset.targets)
indices = np.arange(len(targets))

train_idx, val_idx = train_test_split(
    indices,
    test_size=1 - train_ratio,
    stratify=targets,
    random_state=42
)

train_dataset = Subset(datasets.ImageFolder(root=root_dir, transform=train_transform), train_idx)
val_dataset = Subset(datasets.ImageFolder(root=root_dir, transform=val_transform), val_idx)

train_targets = targets[train_idx]
class_counts = Counter(train_targets)
num_samples = len(train_dataset)

class_weights = {cls: num_samples / count for cls, count in class_counts.items()}
sample_weights = [class_weights[t] for t in train_targets]

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=num_samples, replacement=True)

train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    sampler=sampler,
    num_workers=num_workers)
val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size, 
    shuffle=False,
    num_workers=num_workers)

print("Classes:", full_dataset.classes)
print("Train size:", len(train_dataset))
print("Validation size:", len(val_dataset))
print("Train class counts:", class_counts)

Classes: ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic']
Train size: 36276
Validation size: 9070
Train class counts: Counter({np.int64(1): 12689, np.int64(3): 8522, np.int64(4): 4450, np.int64(6): 2632, np.int64(7): 2421, np.int64(2): 2303, np.int64(5): 1748, np.int64(0): 1511})


# Model Definition

In [19]:
num_classes = len(full_dataset.classes)

model = resnet18(weights="IMAGENET1K_V1")
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss /= val_total
    val_acc = val_correct / val_total

    f1_macro = f1_score(all_labels, all_preds, average='macro')
    print(
        f"Epoch [{epoch+1}/{epochs}] "
        f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
        f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | "
        f"F1 (macro): {f1_macro:.4f}")

print("\nFinal classification report:")
print(classification_report(all_labels, all_preds, target_names=full_dataset.classes))


Epoch [1/10] Train Loss: 0.3508 | Train Acc: 0.8825 | Val Loss: 0.1896 | Val Acc: 0.9405 | F1 (macro): 0.9184
Epoch [2/10] Train Loss: 0.2826 | Train Acc: 0.9051 | Val Loss: 0.1839 | Val Acc: 0.9444 | F1 (macro): 0.9253
Epoch [3/10] Train Loss: 0.2425 | Train Acc: 0.9200 | Val Loss: 0.1483 | Val Acc: 0.9549 | F1 (macro): 0.9408
Epoch [4/10] Train Loss: 0.2150 | Train Acc: 0.9285 | Val Loss: 0.1406 | Val Acc: 0.9551 | F1 (macro): 0.9396
Epoch [5/10] Train Loss: 0.1964 | Train Acc: 0.9337 | Val Loss: 0.1090 | Val Acc: 0.9656 | F1 (macro): 0.9509
Epoch [6/10] Train Loss: 0.1898 | Train Acc: 0.9370 | Val Loss: 0.1201 | Val Acc: 0.9643 | F1 (macro): 0.9512
Epoch [7/10] Train Loss: 0.1753 | Train Acc: 0.9410 | Val Loss: 0.1420 | Val Acc: 0.9526 | F1 (macro): 0.9408
Epoch [8/10] Train Loss: 0.1737 | Train Acc: 0.9422 | Val Loss: 0.1035 | Val Acc: 0.9685 | F1 (macro): 0.9593
Epoch [9/10] Train Loss: 0.1610 | Train Acc: 0.9463 | Val Loss: 0.0975 | Val Acc: 0.9691 | F1 (macro): 0.9562
Epoch [10/

# Exporting the Model

In [23]:
torch.save(model.state_dict(), "model.pth")

In [25]:
dummy_input = torch.randn(1, 3, 224, 224).to(device)
torch.onnx.export(
    model,
    dummy_input,
    "model.onnx",
    input_names=["input"],
    output_names=["output"],
    opset_version=12
)

W1103 22:39:45.899000 17240 Lib\site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 12 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 12).
Failed to convert the model to the target version 12 using the ONNX C API. The model was not modified
Traceback (most recent call last):
  File "d:\Projects\radya\garbage-classification\venv\Lib\site-packages\onnxscript\version_converter\__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Projects\radya\garbage-classification\venv\Lib\site-packages\onnxscript\version_converter\_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
             ^^^^^^^^^^^
  File "d:\Projects\radya\garbage-classification\venv\Lib\site-packages\onnxscript\version_converter\__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 40 of general pattern rewrite rules.


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 18},
            producer_name='pytorch',
            producer_version='2.9.0+cu126',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"input"<FLOAT,[1,3,224,224]>
            ),
            outputs=(
                %"output"<FLOAT,[1,8]>
            ),
            initializers=(
                %"fc.bias"<FLOAT,[8]>{TorchTensor<FLOAT,[8]>(Parameter containing: tensor([ 0.0259,  0.0412,  0.0469, -0.0120,  0.0289,  0.0023, -0.0309, -0.0034], device='cuda:0', requires_grad=True), name='fc.bias')},
                %"conv1.weight"<FLOAT,[64,3,7,7]>{Tensor(...)},
                %"layer1.0.conv1.weight"<FLOAT,[64,64,3,3]>{Tensor(...)},
                %"layer1.0.conv2.weight"<FLOAT,[64,64,3,3]>{Tensor(...)},
                %"layer1.1.conv1.weight"<FLOAT,[64,64,3,3]>{Tensor(...)},
                %"la